In [ ]:
from ftplib import FTP
import ftplib
import time
import tarfile
import os
import datetime
import sys

def ftpconnect(host, username, password):
    ftp = FTP()
    ftp.connect(host, 21)
    ftp.login(username, password)
    return ftp

#从ftp下载文件
def downloadfile(ftp, remotepath, localpath):
    bufsize = 1024
    fp = open(localpath, 'wb')
    ftp.retrbinary('RETR ' + remotepath, fp.write, bufsize)
    ftp.set_debuglevel(0)
    fp.close()


if __name__ == "__main__":
    print('{}'.format('Please Choose Mode: \n1. You Set the Start and End.\n2. Downlaod From What You Have to the Latest'))
    Mode = input()
    today =datetime.date.today()- datetime.timedelta(1)   #获取今天日期
    today = str(today).replace('-','')
    
    if Mode == '1':
        X1 = print("Enter StartDate, example:20170504")
        X2 = print("Enter EndDate, example:20170504")

    elif Mode == '2':
        print('{}'.format('Please Enter the Address of ZZ(郑州) folder Ex: C:/Users/kinsly/Work' ))
        ZZAddress = input()
        os.chdir(ZZAddress) ## Set working directory
        DirectoryList = os.listdir()
        DataDate = []
        ExchangeList = ['ZJ']
        for item in DirectoryList:
            if item[0:2] in ExchangeList:
                DataDate.append(item[2:10])
        LatestDate = max(DataDate) # Find the Latest Date
        X1 = LatestDate
        X2 = today
    else:
        print('{}'.format('You Enter the Wrong Number. Do You Really Understand 英国历史?'))
        sys.exit()
    
    X1 = datetime.datetime.strptime(X1,'%Y%m%d').date() ## 输入两个日期，判断两个日期大小
    X2 = datetime.datetime.strptime(X2,'%Y%m%d').date()
    
    if X1 > X2:
        DateFar = X2
        DateNear = X1
    else:
        DateFar = X1
        DateNear = X2  
    DateList = []
    length = (DateNear - DateFar).days
    for i in range(length+1):
        DateList.append(str(DateFar + datetime.timedelta(i)).replace('-','')) ## 建立一个日期列表
    
    
    #address = ['DL','SQ','ZZ','ZJ','SH','SZ']
    
    DownloadAddress = []
    print("Please Enter the Address You Want to Save.\nDon't use \, Please use/ \n Ex: C:/Users/kinsly/Work")
    x = input()
    ftp = ftpconnect()
    BlankList = []
    for DateItems in DateList:
        address = ['ZJ'] ## 抓取中金所
        NewAddress = []
        DownloadAddress = []
        for items in address:
            NewAddress.append("/" + items + "/" + items + DateItems + ".rar") ## ftp索引路径
            DownloadAddress.append(x + '/' + items + DateItems + '.rar') ## 电脑本地路径
        
        AddressDict = dict(zip(NewAddress,DownloadAddress))
        for k, v in zip(NewAddress,DownloadAddress):
            print('Save    ' + v)
            try:
                downloadfile(ftp, k, v)
            except ftplib.all_errors:
                print(DateItems + ' is not a working day')
                BlankList.append(v)
    ftp.quit()
    for item in BlankList:
        print('Delete  ' + item)
        os.remove(item)
    print('Congrats! We are done!:)\n你太优秀啦！！！')
        
        ## get the lastest date and download data to the latest